In [19]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from keras.models import *
from keras.layers import *
from keras.callbacks import *
from tensorflow import keras

def prepare_data(seq,num):
  x=[]
  y=[]
  for i in range(0,(len(seq)-num),1):
    
    input_ = seq[i:i+num]
    output  = seq[i+num]
    
    x.append(input_)
    y.append(output)
    
  return np.array(x), np.array(y)

data=pd.read_csv('../Utilities\csv\metrics_2024-09-07\metrics.csv')
print(data.head)
cpu_usage = data['value'].values

num=40 # The first 10 mins of traffic
sample = cpu_usage[:num]

x,y= prepare_data(cpu_usage,num)
print(len(x))

ind = int(0.7 * len(x))
x_tr = x[:ind]
y_tr = y[:ind]
x_val=x[ind:]
y_val=y[ind:]

#normalize the inputs
x_scaler= StandardScaler()
x_tr = x_scaler.fit_transform(x_tr)
x_val= x_scaler.transform(x_val)

#reshaping the output for normalization
y_tr=y_tr.reshape(len(y_tr),1)
y_val=y_val.reshape(len(y_val),1)

#normalize the output
y_scaler=StandardScaler()
y_tr = y_scaler.fit_transform(y_tr)[:,0]
y_val = y_scaler.transform(y_val)[:,0]

#reshaping input data
x_tr= x_tr.reshape(x_tr.shape[0],x_tr.shape[1],1)
x_val= x_val.reshape(x_val.shape[0],x_val.shape[1],1)
print(x_tr.shape)

def traffic_prediction_lstm():
    model = Sequential()
    # Bidirectional LSTM layer
    model.add(Bidirectional(LSTM(50, return_sequences=True, dropout=0.2, recurrent_dropout=0.2), input_shape=(num, 1)))
    # Additional LSTM layers
    model.add(LSTM(40, return_sequences=True, dropout=0.2, recurrent_dropout=0.2))
    model.add(LSTM(30, return_sequences=False, dropout=0.2, recurrent_dropout=0.2))
    # Batch Normalization
    model.add(BatchNormalization())
    # Dense layer with ReLU activation
    model.add(Dense(30, activation='relu'))
    # Output layer for regression
    model.add(Dense(1, activation='linear'))
    # Compile the model
    model.compile(optimizer='adam', loss='mse')
    # Learning Rate Scheduler
    lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=5, min_lr=1e-6)
    # Print model summary
    model.summary()
    return model, lr_scheduler


lstm_model, lr_scheduler = traffic_prediction_lstm()
# Early Stopping to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Fit the model
history = lstm_model.fit(x_tr, y_tr,
                    epochs=100,
                    batch_size=32,
                    validation_split=0.2,
                    callbacks=[early_stopping, lr_scheduler])

mse = lstm_model.evaluate(x_val,y_val)
print("Mean Square Error:",mse)

#build a simple model
def compute_moving_average(data):
  pred=[]
  for i in data:
    avg=np.sum(i)/len(i)
    pred.append(avg)
  return np.array(pred)
x_reshaped = x_val.reshape(-1,40)
y_pred = compute_moving_average(x_reshaped)
mse = np.sum ( (y_val - y_pred) **2 ) / (len(y_val))
print("Mean Square Error:",mse)

<bound method NDFrame.head of                timestamp      value
0    07/09/2024 17:00:00  14.312299
1    07/09/2024 17:00:15  10.533495
2    07/09/2024 17:00:30  11.239361
3    07/09/2024 17:00:45  11.239361
4    07/09/2024 17:01:00   9.610936
..                   ...        ...
276  07/09/2024 18:09:00   0.012682
277  07/09/2024 18:09:15   0.011093
278  07/09/2024 18:09:30   0.015599
279  07/09/2024 18:09:45   0.012942
280  07/09/2024 18:10:00   0.015930

[281 rows x 2 columns]>
241
(168, 40, 1)
Model: "sequential_18"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bidirectional_7 (Bidirecti  (None, 40, 100)           20800     
 onal)                                                           
                                                                 
 bidirectional_8 (Bidirecti  (None, 40, 80)            45120     
 onal)                                                           
           